In [1]:
from tensorflow.keras.layers import Dense, Input, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import resnet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
batch_size = 64
width, height, channel = 400, 400, 3
no_epochs = 30
verbosity = 2
input_shape = (width, height, channel)

In [3]:
# ResNet50 모델 사용

pretrained_model = tf.keras.applications.InceptionV3(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=1000
) 

87916544/87910968 [==============================] - 12s 0us/step


In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory('preprocessed_data/train',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory('preprocessed_data/test',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

pretrained_model.trainable=True
set_trainable=False

for layer in pretrained_model.layers:
    if layer.name == 'block7c_project_conv':
        set_trainable=True
    if set_trainable:
        layer.trainable=True
    else:
        layer.trainable=False


model=Sequential()
model.add(pretrained_model)
model.add(MaxPooling2D())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[metrics.AUC(), 'accuracy'])

Found 2603 images belonging to 2 classes.
Found 650 images belonging to 2 classes.


In [5]:
es_callback = EarlyStopping(monitor='val_auc', mode='max', patience=8,
                                              verbose=1, min_delta=0.0001, restore_best_weights=True)

history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples//batch_size,
                    epochs = no_epochs,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//batch_size,
                    callbacks= [es_callback],
                    verbose=verbosity,
                   )

Epoch 1/30
40/40 - 32s - loss: 0.6763 - auc: 0.8836 - accuracy: 0.8106 - val_loss: 0.0908 - val_auc: 0.9978 - val_accuracy: 0.9625
Epoch 2/30
40/40 - 27s - loss: 0.0710 - auc: 0.9972 - accuracy: 0.9811 - val_loss: 0.0371 - val_auc: 1.0000 - val_accuracy: 0.9875
Epoch 3/30
40/40 - 26s - loss: 0.0284 - auc: 0.9999 - accuracy: 0.9953 - val_loss: 0.0163 - val_auc: 1.0000 - val_accuracy: 1.0000
Epoch 4/30
40/40 - 27s - loss: 0.0157 - auc: 1.0000 - accuracy: 0.9988 - val_loss: 0.0103 - val_auc: 1.0000 - val_accuracy: 1.0000
Epoch 5/30
40/40 - 26s - loss: 0.0129 - auc: 1.0000 - accuracy: 0.9996 - val_loss: 0.0105 - val_auc: 1.0000 - val_accuracy: 1.0000
Epoch 6/30
40/40 - 26s - loss: 0.0099 - auc: 1.0000 - accuracy: 1.0000 - val_loss: 0.0063 - val_auc: 1.0000 - val_accuracy: 1.0000
Epoch 7/30
40/40 - 26s - loss: 0.0065 - auc: 1.0000 - accuracy: 1.0000 - val_loss: 0.0050 - val_auc: 1.0000 - val_accuracy: 1.0000
Epoch 8/30
40/40 - 26s - loss: 0.0053 - auc: 1.0000 - accuracy: 0.9996 - val_loss: 

In [6]:
prediction = model.predict(valid_generator)

In [7]:
import numpy as np

print(len(prediction[np.where(prediction > 0.5)]))
print(len(prediction[np.where(prediction < 0.5)]))

312
338


In [8]:
prediction

array([[9.5978212e-01],
       [1.0000000e+00],
       [9.9973375e-01],
       [9.9722052e-01],
       [9.9135405e-01],
       [9.9999940e-01],
       [2.3927973e-03],
       [9.9942052e-01],
       [1.8980922e-03],
       [9.9999976e-01],
       [8.5207932e-03],
       [6.2779650e-06],
       [8.4086470e-03],
       [3.8578782e-02],
       [9.9843019e-01],
       [1.7600658e-03],
       [2.0153518e-03],
       [1.1210397e-02],
       [9.8198617e-01],
       [9.8517078e-01],
       [9.8508471e-01],
       [9.3458146e-01],
       [1.7878975e-03],
       [9.8925740e-01],
       [9.9997318e-01],
       [9.0940291e-01],
       [9.2132926e-01],
       [2.3670371e-03],
       [3.1503849e-03],
       [2.2420166e-03],
       [9.9999988e-01],
       [5.3776592e-01],
       [1.0694873e-02],
       [3.7017602e-03],
       [8.4086470e-03],
       [9.9999964e-01],
       [3.1503849e-03],
       [3.7017602e-03],
       [9.4192672e-01],
       [2.4217388e-02],
       [1.4304147e-04],
       [9.573506

In [9]:
tf.__version__

'2.3.1'

In [10]:
model.evaluate(valid_generator)

11/11 [==============================] - 5s 444ms/step - loss: 0.0368 - auc: 1.0000 - accuracy: 0.9877


[0.036809444427490234, 1.0, 0.9876922965049744]